In [4]:
using ArchGDAL

In [114]:
ArchGDAL.registerdrivers() do
    ArchGDAL.read("data/gpw-v4-population-density-rev11_2020_2pt5_min_tif/gpw_v4_population_density_rev11_2020_2pt5_min.tif") do dataset
        # Get the band of data corresponding to population density per ~5km square of the Earth's surface
        band = ArchGDAL.getband(dataset, 1)
        print(band)
        min_lat = 8.040682
        max_lat = 11.2195684
        min_lon = -85.956896
        max_lon = -82.5060208
        band_width = ArchGDAL.width(band)
        band_height = ArchGDAL.height(band)        
        max_lat_ind = ceil((90 - min_lat) * band_height / 180)
        min_lat_ind = min(ceil((90 - max_lat) * band_height / 180), max_lat_ind - 1)
        min_lon_ind = ceil((min_lon + 180) * band_width / 360)
        max_lon_ind = max(ceil((max_lon + 180) * band_width / 360), min_lon_ind + 1)
        rows = UnitRange{Int}(min_lat_ind, max_lat_ind)
        cols = UnitRange{Int}(min_lon_ind, max_lon_ind)
        bounding_box_contents = ArchGDAL.read(band, rows, cols)
    end
end

[GA_ReadOnly] Band 1 (Gray): 8640 x 4320 (Float32)
    blocksize: 8640×1, nodata: -3.4028230607370965e38, units: 1.0px + 0.0
    overviews: 

83×78 Array{Float32,2}:
  -3.40282e38  -3.40282e38  -3.40282e38  …  -3.40282e38  -3.40282e38
  42.0993      -3.40282e38  -3.40282e38     -3.40282e38  -3.40282e38
  42.0993      42.0993      -3.40282e38     -3.40282e38  -3.40282e38
  42.0993      42.0993      42.0993         -3.40282e38  -3.40282e38
  42.0993      42.0993      42.0993         -3.40282e38  -3.40282e38
  55.0016      42.0993      42.0993      …  -3.40282e38  -3.40282e38
 144.736       49.1092      43.5223         -3.40282e38  -3.40282e38
  73.7236      48.3599      32.9325         -3.40282e38  -3.40282e38
  49.7691      30.1995      26.203          -3.40282e38  -3.40282e38
  51.963       31.786       22.7825         -3.40282e38  -3.40282e38
  53.1764      50.067       11.4859      …  -3.40282e38  -3.40282e38
   0.0         53.1764      28.0372         -3.40282e38  -3.40282e38
   0.0         53.1764      44.3192         -3.40282e38  -3.40282e38
   ⋮                                     ⋱                          
  -3.40282